In [1]:
import numpy as np
import cv2
import tensorflow as tf
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import RepeatVector
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

import time as t
from statistics import mean

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=11264)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    #p = LSTM(200, activation=None)(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model


In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='relu')(b)
    d = RepeatVector(vector_shape[0])(d)
    d = LSTM(200, activation='relu', return_sequences=True)(d)
    # output
    out_layer = Dense(3)(d)

    # define model input & output
    model = Model([in_image, in_lat], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	return [X1, X2]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	trainA, trainB = dataset
	# choose random instances
	ix = randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_realA[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_fakeB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_realA[i]* 255).astype(np.uint8), X_realB[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [13]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=25, n_batch=3):
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(dataset[0]) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    #for timining
    #n_steps = 25
    times = []
    # manually enumerate epochs
    for i in range(n_steps):
        #start timer
        start = t.time()
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_batch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        X_lat = generate_latent_points(latent_dim, n_batch)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_realA, X_lat], [y_real, X_realB])
        #end timer
        end = t.time()
        #print(f'time for itr: {end-start} sec')
        times.append(end-start)
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(bat_per_epo) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')
    print(f'average time per itr: {mean(times)}')

In [14]:
%%time
# load image data
#dataset = load_real_samples('0000.npz')
dataset = load_real_samples('dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3)
CPU times: user 22.3 s, sys: 611 ms, total: 22.9 s
Wall time: 22.9 s


In [15]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
print(image_shape)
print(vector_shape)

(256, 256, 3)
(40, 3)


In [16]:
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)

In [17]:
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [ ]:
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.504] d2[3.070] g[401.646]
>2, d1[0.015] d2[0.205] g[536.199]
>3, d1[0.085] d2[0.425] g[325.522]
>4, d1[0.281] d2[0.331] g[339.893]
>5, d1[0.024] d2[0.369] g[550.143]
>6, d1[0.080] d2[0.190] g[323.163]
>7, d1[0.010] d2[0.280] g[629.016]
>8, d1[0.003] d2[0.202] g[501.378]
>9, d1[0.030] d2[0.150] g[583.758]
>10, d1[0.001] d2[0.325] g[671.937]
>11, d1[0.008] d2[0.117] g[362.904]
>12, d1[0.380] d2[0.747] g[268.114]
>13, d1[0.069] d2[1.122] g[211.389]
>14, d1[0.650] d2[0.448] g[151.428]
>15, d1[0.800] d2[0.159] g[400.117]
>16, d1[0.251] d2[0.185] g[493.186]
>17, d1[0.083] d2[0.270] g[341.716]
>18, d1[0.211] d2[0.390] g[296.626]
>19, d1[0.346] d2[0.259] g[222.854]
>20, d1[0.038] d2[0.677] g[332.742]
>21, d1[0.341] d2[0.447] g[1234.405]
>22, d1[0.204] d2[0.691] g[644.102]
>23, d1[0.239] d2[0.344] g[401.373]
>24, d1[0.298] d2[0.334] g[337.138]
>25, d1[0.346] d2[0.508] g[275.295]
>26, d1[0.107] d2[0.228] g[411.384]
>27, d1[0.135] d2[0.420] g[287.745]
>28, d1[0.146] d2[0.216] g[414.417]


>227, d1[0.090] d2[0.118] g[242.683]
>228, d1[0.022] d2[0.011] g[187.319]
>229, d1[0.012] d2[0.089] g[164.334]
>230, d1[0.002] d2[0.001] g[92.481]
>231, d1[0.011] d2[0.005] g[185.774]
>232, d1[0.025] d2[0.003] g[279.143]
>233, d1[0.000] d2[0.001] g[145.509]
>234, d1[0.002] d2[0.002] g[210.784]
>235, d1[0.004] d2[0.018] g[169.464]
>236, d1[0.001] d2[0.001] g[159.463]
>237, d1[0.002] d2[0.001] g[150.288]
>238, d1[0.010] d2[0.003] g[154.971]
>239, d1[0.023] d2[0.011] g[159.131]
>240, d1[0.025] d2[0.005] g[191.491]
>241, d1[0.011] d2[0.006] g[160.261]
>242, d1[0.004] d2[0.000] g[314.117]
>243, d1[0.017] d2[0.005] g[111.350]
>244, d1[0.003] d2[0.008] g[79.567]
>245, d1[0.002] d2[0.000] g[230.888]
>246, d1[0.007] d2[0.006] g[328.122]
>247, d1[0.017] d2[0.002] g[201.200]
>248, d1[0.025] d2[0.005] g[163.388]
>249, d1[0.006] d2[0.001] g[169.102]
>250, d1[0.001] d2[0.000] g[113.470]
>251, d1[0.000] d2[0.001] g[288.099]
>252, d1[0.011] d2[0.000] g[189.657]
>253, d1[0.006] d2[0.000] g[195.300]
>25

>450, d1[0.363] d2[0.328] g[137.410]
>451, d1[0.358] d2[0.379] g[239.745]
>452, d1[0.345] d2[0.288] g[178.720]
>453, d1[0.364] d2[0.319] g[125.326]
>454, d1[0.390] d2[0.297] g[169.122]
>455, d1[0.366] d2[0.314] g[85.890]
>456, d1[0.398] d2[0.354] g[199.947]
>457, d1[0.295] d2[0.311] g[144.197]
>458, d1[0.331] d2[0.326] g[246.651]
>459, d1[0.261] d2[0.376] g[239.696]
>460, d1[0.347] d2[0.449] g[241.041]
>461, d1[0.343] d2[0.492] g[112.939]
>462, d1[0.299] d2[0.338] g[130.272]
>463, d1[0.304] d2[0.338] g[228.608]
>464, d1[0.500] d2[0.339] g[97.616]
>465, d1[0.351] d2[0.331] g[178.059]
>466, d1[0.409] d2[0.331] g[185.084]
>467, d1[0.306] d2[0.413] g[109.490]
>468, d1[0.283] d2[0.384] g[166.562]
>469, d1[0.370] d2[0.442] g[233.762]
>470, d1[0.403] d2[0.351] g[136.979]
>471, d1[0.292] d2[0.384] g[118.946]
>472, d1[0.206] d2[0.310] g[312.477]
>473, d1[0.285] d2[0.363] g[318.006]
>474, d1[0.282] d2[0.482] g[120.768]
>475, d1[0.296] d2[0.370] g[162.520]
>476, d1[0.390] d2[0.328] g[46.638]
>477

>674, d1[0.307] d2[0.376] g[86.515]
>675, d1[0.339] d2[0.437] g[115.303]
>676, d1[0.335] d2[0.422] g[198.047]
>677, d1[0.442] d2[0.356] g[100.462]
>678, d1[0.339] d2[0.415] g[142.267]
>679, d1[0.328] d2[0.352] g[103.409]
>680, d1[0.329] d2[0.331] g[155.800]
>681, d1[0.279] d2[0.398] g[79.765]
>682, d1[0.306] d2[0.347] g[265.666]
>683, d1[0.296] d2[0.384] g[274.720]
>684, d1[0.303] d2[0.384] g[249.588]
>685, d1[0.248] d2[0.376] g[99.302]
>686, d1[0.332] d2[0.384] g[143.009]
>687, d1[0.313] d2[0.370] g[77.287]
>688, d1[0.285] d2[0.371] g[84.745]
>689, d1[0.378] d2[0.304] g[131.889]
>690, d1[0.339] d2[0.378] g[119.296]
>691, d1[0.430] d2[0.317] g[325.719]
>692, d1[0.436] d2[0.345] g[182.526]
>693, d1[0.332] d2[0.384] g[161.140]
>694, d1[0.314] d2[0.283] g[149.442]
>695, d1[0.356] d2[0.340] g[85.013]
>696, d1[0.349] d2[0.404] g[186.704]
>697, d1[0.363] d2[0.392] g[99.826]
>698, d1[0.356] d2[0.403] g[176.631]
>699, d1[0.298] d2[0.304] g[174.644]
>700, d1[0.297] d2[0.348] g[84.124]
>701, d1[

>898, d1[0.412] d2[0.301] g[167.904]
>899, d1[0.356] d2[0.337] g[56.273]
>900, d1[0.401] d2[0.365] g[103.519]
>901, d1[0.307] d2[0.311] g[134.905]
>902, d1[0.348] d2[0.321] g[88.403]
>903, d1[0.324] d2[0.353] g[221.385]
>904, d1[0.341] d2[0.329] g[167.178]
>905, d1[0.331] d2[0.344] g[223.582]
>906, d1[0.334] d2[0.404] g[192.851]
>907, d1[0.317] d2[0.366] g[102.708]
>908, d1[0.375] d2[0.354] g[183.604]
>909, d1[0.382] d2[0.358] g[245.473]
>910, d1[0.381] d2[0.376] g[238.199]
>911, d1[0.374] d2[0.363] g[86.093]
>912, d1[0.338] d2[0.362] g[188.032]
>913, d1[0.368] d2[0.371] g[141.113]
>914, d1[0.384] d2[0.338] g[212.057]
>915, d1[0.379] d2[0.332] g[96.035]
>916, d1[0.370] d2[0.395] g[307.704]
>917, d1[0.359] d2[0.338] g[275.625]
>918, d1[0.373] d2[0.343] g[138.880]
>919, d1[0.370] d2[0.348] g[148.350]
>920, d1[0.347] d2[0.315] g[140.927]
>921, d1[0.441] d2[0.315] g[80.094]
>922, d1[0.370] d2[0.321] g[151.372]
>923, d1[0.380] d2[0.396] g[106.243]
>924, d1[0.294] d2[0.314] g[163.341]
>925, 

...

>41069, d1[0.347] d2[0.339] g[16.708]
>41070, d1[0.348] d2[0.341] g[39.604]
>41071, d1[0.367] d2[0.333] g[8.253]
>41072, d1[0.354] d2[0.338] g[17.257]
>41073, d1[0.359] d2[0.358] g[26.279]
>41074, d1[0.352] d2[0.355] g[13.613]
>41075, d1[0.366] d2[0.345] g[26.712]
>41076, d1[0.361] d2[0.344] g[23.605]
>41077, d1[0.358] d2[0.345] g[30.460]
>41078, d1[0.342] d2[0.336] g[17.336]
>41079, d1[0.340] d2[0.352] g[16.879]
>41080, d1[0.366] d2[0.343] g[22.888]
>41081, d1[0.352] d2[0.351] g[13.643]
>41082, d1[0.342] d2[0.350] g[19.626]
>41083, d1[0.347] d2[0.351] g[33.338]
>41084, d1[0.329] d2[0.363] g[33.520]
>41085, d1[0.357] d2[0.359] g[17.994]
>41086, d1[0.333] d2[0.354] g[24.109]
>41087, d1[0.318] d2[0.380] g[30.312]
>41088, d1[0.344] d2[0.370] g[27.042]
>41089, d1[0.353] d2[0.341] g[22.932]
>41090, d1[0.335] d2[0.336] g[43.737]
>41091, d1[0.348] d2[0.364] g[18.578]
>41092, d1[0.340] d2[0.351] g[12.847]
>41093, d1[0.342] d2[0.385] g[22.385]
>41094, d1[0.331] d2[0.343] g[17.151]
>41095, d1[0.

>41286, d1[0.347] d2[0.369] g[20.373]
>41287, d1[0.343] d2[0.331] g[19.597]
>41288, d1[0.365] d2[0.358] g[21.538]
>41289, d1[0.309] d2[0.357] g[15.514]
>41290, d1[0.314] d2[0.323] g[23.928]
>41291, d1[0.372] d2[0.331] g[12.245]
>41292, d1[0.329] d2[0.337] g[26.743]
>41293, d1[0.365] d2[0.368] g[15.962]
>41294, d1[0.398] d2[0.351] g[18.543]
>41295, d1[0.313] d2[0.300] g[15.620]
>41296, d1[0.409] d2[0.347] g[17.016]
>41297, d1[0.381] d2[0.382] g[25.496]
>41298, d1[0.346] d2[0.382] g[21.053]
>41299, d1[0.289] d2[0.360] g[15.126]
>41300, d1[0.352] d2[0.369] g[20.341]
>41301, d1[0.337] d2[0.329] g[18.505]
>41302, d1[0.350] d2[0.367] g[11.990]
>41303, d1[0.366] d2[0.332] g[18.881]
>41304, d1[0.348] d2[0.349] g[16.396]
>41305, d1[0.346] d2[0.341] g[16.702]
>41306, d1[0.373] d2[0.361] g[20.307]
>41307, d1[0.347] d2[0.325] g[14.832]
>41308, d1[0.357] d2[0.375] g[29.160]
>41309, d1[0.368] d2[0.349] g[16.705]
>41310, d1[0.336] d2[0.332] g[13.200]
>41311, d1[0.322] d2[0.369] g[20.096]
>41312, d1[0

>41502, d1[0.350] d2[0.341] g[10.676]
>41503, d1[0.355] d2[0.347] g[22.687]
>41504, d1[0.352] d2[0.342] g[16.015]
>41505, d1[0.379] d2[0.332] g[27.131]
>41506, d1[0.363] d2[0.334] g[18.076]
>41507, d1[0.326] d2[0.329] g[40.137]
>41508, d1[0.364] d2[0.357] g[15.690]
>41509, d1[0.361] d2[0.336] g[30.674]
>41510, d1[0.355] d2[0.330] g[31.140]
>41511, d1[0.352] d2[0.347] g[21.207]
>41512, d1[0.376] d2[0.314] g[16.762]
>41513, d1[0.348] d2[0.319] g[28.553]
>41514, d1[0.349] d2[0.323] g[24.930]
>41515, d1[0.369] d2[0.346] g[15.907]
>41516, d1[0.351] d2[0.345] g[17.707]
>41517, d1[0.357] d2[0.354] g[16.257]
>41518, d1[0.344] d2[0.337] g[20.879]
>41519, d1[0.355] d2[0.329] g[16.598]
>41520, d1[0.371] d2[0.358] g[21.950]
>41521, d1[0.352] d2[0.386] g[26.515]
>41522, d1[0.346] d2[0.328] g[10.726]
>41523, d1[0.329] d2[0.378] g[25.613]
>41524, d1[0.340] d2[0.369] g[19.016]
>41525, d1[0.338] d2[0.366] g[21.559]
>41526, d1[0.341] d2[0.377] g[46.720]
>41527, d1[0.331] d2[0.359] g[15.830]
>41528, d1[0

>41719, d1[0.325] d2[0.333] g[38.188]
>41720, d1[0.357] d2[0.356] g[16.243]
>41721, d1[0.332] d2[0.361] g[18.687]
>41722, d1[0.313] d2[0.364] g[17.456]
>41723, d1[0.353] d2[0.388] g[9.064]
>41724, d1[0.337] d2[0.365] g[12.483]
>41725, d1[0.348] d2[0.371] g[13.582]
>41726, d1[0.335] d2[0.372] g[9.135]
>41727, d1[0.357] d2[0.365] g[27.775]
>41728, d1[0.349] d2[0.365] g[24.960]
>41729, d1[0.340] d2[0.359] g[6.096]
>41730, d1[0.358] d2[0.366] g[16.041]
>41731, d1[0.339] d2[0.368] g[14.632]
>41732, d1[0.335] d2[0.350] g[14.718]
>41733, d1[0.367] d2[0.342] g[23.746]
>41734, d1[0.355] d2[0.351] g[15.287]
>41735, d1[0.332] d2[0.382] g[28.241]
>41736, d1[0.343] d2[0.351] g[14.137]
>41737, d1[0.349] d2[0.340] g[18.255]
>41738, d1[0.345] d2[0.350] g[21.656]
>41739, d1[0.357] d2[0.363] g[21.814]
>41740, d1[0.355] d2[0.334] g[29.424]
>41741, d1[0.336] d2[0.345] g[22.791]
>41742, d1[0.337] d2[0.346] g[24.173]
>41743, d1[0.329] d2[0.360] g[27.737]
>41744, d1[0.330] d2[0.350] g[16.516]
>41745, d1[0.35

>41936, d1[0.349] d2[0.346] g[10.024]
>41937, d1[0.364] d2[0.317] g[28.751]
>41938, d1[0.348] d2[0.334] g[12.606]
>41939, d1[0.332] d2[0.380] g[33.097]
>41940, d1[0.340] d2[0.373] g[33.879]
>41941, d1[0.370] d2[0.327] g[10.423]
>41942, d1[0.350] d2[0.364] g[13.540]
>41943, d1[0.345] d2[0.342] g[28.716]
>41944, d1[0.325] d2[0.335] g[16.824]
>41945, d1[0.333] d2[0.350] g[10.858]
>41946, d1[0.362] d2[0.344] g[37.768]
>41947, d1[0.331] d2[0.350] g[14.136]
>41948, d1[0.341] d2[0.337] g[20.833]
>41949, d1[0.355] d2[0.354] g[34.415]
>41950, d1[0.323] d2[0.349] g[27.578]
>41951, d1[0.331] d2[0.348] g[22.214]
>41952, d1[0.392] d2[0.369] g[21.379]
>41953, d1[0.366] d2[0.334] g[10.377]
>41954, d1[0.340] d2[0.314] g[13.376]
>41955, d1[0.344] d2[0.338] g[18.619]
>41956, d1[0.333] d2[0.409] g[18.806]
>41957, d1[0.335] d2[0.346] g[14.963]
>41958, d1[0.322] d2[0.357] g[23.338]
>41959, d1[0.303] d2[0.369] g[12.706]
>41960, d1[0.282] d2[0.390] g[17.653]
>41961, d1[0.305] d2[0.367] g[22.423]
>41962, d1[0

In [ ]:
#end the tensorflow session
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
#for cuda to release GPU memory
from numba import cuda
cuda.select_device(0)
cuda.close()